In [3]:
import os
import shutil
import subprocess
import xarray as xr
# import rasterio
# import rasterio.plot

In [4]:
date = "20231028"
dir_path = "storage/" + date
if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
os.mkdir(dir_path)

In [5]:
ds = xr.open_dataset("http://182.16.248.173:8080/dods/INA-NWP/2023102800/2023102800-d01-asim")

c:\Users\falbas\miniconda3\Lib\site-packages\xarray\coding\times.py:167: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [6]:
time = ds['time']
lev = ds['lev']
lat = ds['lat']
lon = ds['lon']
u = ds['u']
v = ds['v']

rainc = ds['rainc']
rainsh = ds['rainsh']
rainnc = ds['rainnc']
tc = ds['tc']
psfc = ds['psfc']
rh = ds['rh']

In [12]:
# make uv wind component ascii grid
ncols = ds.dims.get('lon')
nrows = ds.dims.get('lat')
xllcorner = lon[0].values
yllcorner = lat[-1].values
cellsize = abs(lat[0].values-lat[1].values)
NODATA_value = -9999
u_grid = u[0][0].values
v_grid = u[0][0].values

In [13]:
with open(f'{dir_path}/U.asc', 'w') as file:
    file.write(f'ncols {ncols}\n')
    file.write(f'nrows {nrows}\n')
    file.write(f'xllcorner {xllcorner}\n')
    file.write(f'yllcorner {yllcorner}\n')
    file.write(f'cellsize {cellsize}\n')
    file.write(f'NODATA_value {NODATA_value}\n')
    for i in u_grid:
        for j in i:
            file.write(f'{j} ')
        file.write('\n')

In [14]:
with open(f'{dir_path}/V.asc', 'w') as file:
    file.write(f'ncols {ncols}\n')
    file.write(f'nrows {nrows}\n')
    file.write(f'xllcorner {xllcorner}\n')
    file.write(f'yllcorner {yllcorner}\n')
    file.write(f'cellsize {cellsize}\n')
    file.write(f'NODATA_value {NODATA_value}\n')
    for i in v_grid:
        for j in i:
            file.write(f'{j} ')
        file.write('\n')

In [15]:
# make tiff images
tc_tiff = f"{dir_path}/tc.tiff"
psfc_tiff = f"{dir_path}/psfc.tiff"
rh_tiff = f"{dir_path}/rh.tiff"
rainc_tiff = f"{dir_path}/rainc.tiff"
rainsh_tiff = f"{dir_path}/rainsh.tiff"
rainnc_tiff = f"{dir_path}/rainnc.tiff"

_tc = tc[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
_tc.rio.crs
_tc.rio.write_crs("epsg:4326", inplace=True)
_tc.rio.to_raster(tc_tiff)

_psfc = psfc[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
_psfc.rio.crs
_psfc.rio.write_crs("epsg:4326", inplace=True)
_psfc.rio.to_raster(psfc_tiff)

_rh = rh[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
_rh.rio.crs
_rh.rio.write_crs("epsg:4326", inplace=True)
_rh.rio.to_raster(rh_tiff)

# _rainc = rainc[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
# _rainc.rio.crs
# _rainc.rio.write_crs("epsg:4326", inplace=True)
# _rainc.rio.to_raster(rainc_tiff)

# _rainsh = rainsh[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
# _rainsh.rio.crs
# _rainsh.rio.write_crs("epsg:4326", inplace=True)
# _rainsh.rio.to_raster(rainsh_tiff)

# _rainnc = rainnc[0][0].rio.set_spatial_dims(x_dim="lon", y_dim="lat")
# _rainnc.rio.crs
# _rainnc.rio.write_crs("epsg:4326", inplace=True)
# _rainnc.rio.to_raster(rainnc_tiff)

In [35]:
# convert gray tiff to rgb tiff
tc_rgb_tiff = f"{dir_path}/tc_rgb.tiff"
psfc_rgb_tiff = f"{dir_path}/psfc_rgb.tiff"
rh_rgb_tiff = f"{dir_path}/rh_rgb.tiff"
rainc_rgb_tiff = f"{dir_path}/rainc_rgb.tiff"
rainsh_rgb_tiff = f"{dir_path}/rainsh_rgb.tiff"
rainnc_rgb_tiff = f"{dir_path}/rainnc_rgb.tiff"

subprocess.run([r"gdaldem.exe", "color-relief", tc_tiff, "tc_color.txt", tc_rgb_tiff])
# subprocess.run([r"gdaldem.exe", "color-relief", psfc_tiff, "color.txt", psfc_rgb_tiff])
# subprocess.run([r"gdaldem.exe", "color-relief", rh_tiff, "color.txt", rh_rgb_tiff])
# subprocess.run([r"gdaldem.exe", "color-relief", rainc_tiff, "color.txt", rainc_rgb_tiff])
# subprocess.run([r"gdaldem.exe", "color-relief", rainsh_tiff, "color.txt", rainsh_rgb_tiff])
# subprocess.run([r"gdaldem.exe", "color-relief", rainnc_tiff, "color.txt", rainnc_rgb_tiff])

CompletedProcess(args=['gdaldem.exe', 'color-relief', 'storage/20231028/tc.tiff', 'color2.txt', 'storage/20231028/tc_rgb.tiff'], returncode=0)

In [51]:
# tc_img = rasterio.open(tc_rgb_tiff)
# rasterio.plot.show(tc_img, )
# tc_img.close()

tc_img = ImageOps.flip(Image.open(tc_rgb_tiff))
tc_img = tc_img.save(tc_rgb_tiff)
tc_img = Image.open(tc_rgb_tiff)
display(tc_img)
# rasterio.plot.show(tc_img)
# tc_img.show()

# psfc_img = rasterio.open(psfc_rgb_tiff)
# rasterio.plot.show(psfc_img)
# psfc_img.close()

# rh_img = rasterio.open(rh_rgb_tiff)
# rasterio.plot.show(rh_img)
# rh_img.close()

# rainc_img = rasterio.open(rainc_rgb_tiff)
# rasterio.plot.show(rainc_img)
# rainc_img.close()

# rainsh_img = rasterio.open(rainsh_rgb_tiff)
# rasterio.plot.show(rainsh_img)
# rainsh_img.close()

# rainnc_img = rasterio.open(rainnc_rgb_tiff)
# rasterio.plot.show(rainnc_img)
# rainnc_img.close()

TypeError: 'dict' object is not callable

In [7]:
subprocess.run(["python", "C:/Users/falbas/miniconda3/Scripts/gdal2tiles.py", "--zoom=5-7", f"{dir_path}/temp_rgb.tiff", "tiles/baselayer2"])

CompletedProcess(args=['python', 'C:/Users/falbas/miniconda3/Scripts/gdal2tiles.py', '--zoom=5-7', 'storage/20231028/temp_rgb.tiff', 'tiles/baselayer2'], returncode=0)